<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leer el contenido


In [31]:
import pandas as pd


# Cargar los datos
df = pd.read_csv('train.csv')


Mostrar el contenido en una tabla


In [32]:
df

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106101,378363,20203,GERENCIA EN SISTEMAS DE INFORMACION EN SALUD,ANTIOQUIA,No pagó matrícula,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,medio-alto,0.312,0.220,0.285,0.254
106102,393450,20203,LICENCIATURA EN INFORMATICA Y MEDIOS AUDIOVISU...,CORDOBA,Menos de 500 mil,0,Estrato 2,Si,Secundaria (Bachillerato) incompleta,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.207,0.285,0.259,0.309
106103,636321,20195,LICENCIATURA EN ETNOEDUCACION,BOGOTÁ,No pagó matrícula,Más de 30 horas,Estrato 1,No,Primaria completa,No,...,N,Si,No,No,Primaria completa,medio-bajo,0.162,0.417,0.254,0.300
106104,559355,20203,FINANZAS Y COMERCIO INTERNACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 2,Si,Educación profesional completa,NaN,...,N,No,Si,Si,Técnica o tecnológica incompleta,alto,0.207,0.316,0.270,0.301


In [33]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", categorical_cols)
print("Numéricas:", numerical_cols)

Categóricas: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']
Numéricas: ['ID', 'PERIODO', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


In [34]:
# Aplicar one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)



# Mostrar el DataFrame resultante
df_encoded

,ID,PERIODO,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,904256,20212,0.322,0.208,0.310,0.267,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1,645256,20212,0.311,0.215,0.292,0.264,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
2,308367,20203,0.297,0.214,0.305,0.264,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
3,470353,20195,0.485,0.172,0.252,0.190,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,989032,20212,0.316,0.232,0.285,0.294,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106101,378363,20203,0.312,0.220,0.285,0.254,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
106102,393450,20203,0.207,0.285,0.259,0.309,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
106103,636321,20195,0.162,0.417,0.254,0.300,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
106104,559355,20203,0.207,0.316,0.270,0.301,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [36]:
filas_antes = df_encoded.shape[0]

# Eliminar filas con valores faltantes
df_encoded = df_encoded.dropna()

# Contar filas después de eliminar
filas_despues = df_encoded.shape[0]

# Calcular cuántas se eliminaron
filas_eliminadas = filas_antes - filas_despues

print(f"Filas eliminadas por contener NaN: {filas_eliminadas}")

df_encoded

Filas eliminadas por contener NaN: 0


,ID,PERIODO,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,904256,20212,0.322,0.208,0.310,0.267,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1,645256,20212,0.311,0.215,0.292,0.264,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
2,308367,20203,0.297,0.214,0.305,0.264,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
3,470353,20195,0.485,0.172,0.252,0.190,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,989032,20212,0.316,0.232,0.285,0.294,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106100,441656,20183,0.330,0.234,0.285,0.291,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
106101,378363,20203,0.312,0.220,0.285,0.254,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
106102,393450,20203,0.207,0.285,0.259,0.309,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
106103,636321,20195,0.162,0.417,0.254,0.300,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
